# Stage 1: Golden Sets — Your Foundation for AI Quality

---

## 🎯 What You'll Learn

By the end of this walkthrough, you'll understand:

1. **What golden sets are** and why they're the foundation of every eval framework
2. **How golden sets work** behind the scenes — what actually happens when you run a test
3. **How to write effective test cases** that catch real issues
4. **Where golden sets fit** in the maturity of an eval framework

---

## 📚 Understanding Golden Sets

### What Are Golden Sets?

**Golden sets are curated input/output pairs that define what "correct" looks like for your AI system.** They are the ground truth—manually verified examples that establish baseline correctness.

Think of golden sets like unit tests for traditional software:

| Traditional Software | AI System (Golden Sets) |
|---------------------|------------------------|
| `assertEqual(add(2, 3), 5)` | Query: "What's our PTO policy?" |
| Expected output: `5` | Must contain: "15 days", "annual" |
| Exact match required | Semantic match + constraints |

### Why "Golden"?

The term "golden" comes from "golden master" testing—a technique where you capture a known-good output and compare future outputs against it. These are your **gold standard** reference answers.

### Why Start Here?

Golden sets are Stage 1 because they provide:

1. **Ground truth establishment** — You manually verify these are correct
2. **Fast regression detection** — Run in minutes, catch breakages immediately  
3. **Easy debugging** — When one fails, you know exactly what went wrong
4. **Team alignment** — Everyone agrees on what "correct" means

Without golden sets, you're flying blind—you don't actually know if your agent is working correctly.

---

## 🏗️ Where Golden Sets Fit: The Eval Maturity Model

```
┌─────────────────────────────────────────────────────────────────┐
│                    EVAL FRAMEWORK MATURITY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Stage 5: Experiments      ← Compare configurations            │
│      ▲                                                          │
│  Stage 4: Rubrics          ← Multi-dimensional scoring         │
│      ▲                                                          │
│  Stage 3: Replay Harnesses ← Reproducibility + rich metrics    │
│      ▲                                                          │
│  Stage 2: Labeled Scenarios← Coverage mapping                  │
│      ▲                                                          │
│  ★ Stage 1: GOLDEN SETS ★  ← Baseline correctness (YOU ARE HERE)│
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Golden sets are the foundation.** Every subsequent stage builds on this. If your golden sets are weak, everything above will be unreliable.

## 🔍 Behind the Scenes: How Golden Sets Work

Let's trace what happens when you run a golden set evaluation:

```
┌──────────────────────────────────────────────────────────────────────┐
│                    GOLDEN SET EVALUATION FLOW                         │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│  1. LOAD TEST CASE                                                    │
│     ┌─────────────────────────────────────────────────────────┐      │
│     │ id: gs-001                                               │      │
│     │ query: "What is our remote work policy?"                 │      │
│     │ expected_tools: [vector_search]                          │      │
│     │ must_contain: ["remote", "core hours", "$500"]           │      │
│     └─────────────────────────────────────────────────────────┘      │
│                            │                                          │
│                            ▼                                          │
│  2. RUN THE AGENT                                                     │
│     ┌─────────────────────────────────────────────────────────┐      │
│     │ ask_acme_with_trace(query)                               │      │
│     │   → LLM decides which tool to use                        │      │
│     │   → Calls vector_search("remote work policy")            │      │
│     │   → Retrieves remote_work_policy.md                      │      │
│     │   → LLM generates response from retrieved docs           │      │
│     └─────────────────────────────────────────────────────────┘      │
│                            │                                          │
│                            ▼                                          │
│  3. CAPTURE TRACE                                                     │
│     ┌─────────────────────────────────────────────────────────┐      │
│     │ trace = {                                                │      │
│     │   "response": "Our remote work policy includes...",     │      │
│     │   "tool_calls": [{"tool": "vector_search", ...}]         │      │
│     │ }                                                        │      │
│     └─────────────────────────────────────────────────────────┘      │
│                            │                                          │
│                            ▼                                          │
│  4. RUN CHECKS                                                        │
│     ┌─────────────────────────────────────────────────────────┐      │
│     │ ✓ Tool Check:     Is vector_search in actual_tools?      │      │
│     │ ✓ Source Check:   Is remote_work_policy.md mentioned?    │      │
│     │ ✓ Content Check:  Are "remote", "core hours" found?      │      │
│     │ ✓ Negative Check: Is "I don't know" absent?              │      │
│     └─────────────────────────────────────────────────────────┘      │
│                            │                                          │
│                            ▼                                          │
│  5. REPORT RESULT                                                     │
│     ┌─────────────────────────────────────────────────────────┐      │
│     │ gs-001: PASS ✓                                           │      │
│     │   Tools: ✓  Sources: ✓  Content: ✓  Negative: ✓          │      │
│     └─────────────────────────────────────────────────────────┘      │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

**Key insight:** We're not checking if the response is "exactly right"—we're checking if it meets a set of **constraints** that define correctness. This is essential for AI systems where the exact wording will vary.

In [2]:
import sys
sys.path.insert(0, "../setup_agent")

import yaml
from pathlib import Path
from evaluator import run_test_case, load_golden_set, check_tools, check_must_contain

## 💻 Hands-On: Understanding Golden Set Structure

Now let's see this in action. First, let's examine how a golden set test case is structured:

### Anatomy of a Test Case

Each test case defines constraints that a correct response must satisfy:

| Field | Purpose | Example |
|-------|---------|---------|
| `id` | Unique identifier for tracking | `gs-001` |
| `query` | The input question | "What is our remote work policy?" |
| `expected_tools` | Which tools should be used | `["vector_search"]` |
| `expected_sources` | Which documents should be cited | `["remote_work_policy.md"]` |
| `must_contain` | Keywords that must appear | `["remote", "core hours"]` |
| `must_not_contain` | Phrases that indicate failure | `["I don't know"]` |

In [3]:
# Load the golden set
test_cases = load_golden_set()
print(f"Loaded {len(test_cases)} test cases\n")

# Look at the first test case
first_case = test_cases[0]
print("Example test case:")
print("-" * 40)
for key, value in first_case.items():
    print(f"{key}: {value}")

Loaded 15 test cases

Example test case:
----------------------------------------
id: gs-001
category: vector_search
query: What is our remote work policy?
expected_tools: ['vector_search']
expected_sources: ['remote_work_policy.md']
must_contain: ['remote', 'core hours', '10']
must_not_contain: ["I don't know", 'I cannot find']


## 🧪 Running a Single Test Case

Now let's run one test case and trace exactly what happens. Watch how the evaluator:
1. Sends the query to the agent
2. Captures the response and tool calls
3. Runs each check against the response
4. Produces a pass/fail result

In [4]:
# Run the first test case
result = run_test_case(test_cases[0], verbose=True)

print(f"Test ID: {result.id}")
print(f"Query: {result.query}")
print(f"Passed: {result.passed}")
print(f"Tools Used: {result.tools_used}")
print(f"\nChecks:")
print(f"  - Tool Check: {result.tool_check}")
print(f"  - Source Check: {result.source_check}")
print(f"  - Content Check: {result.content_check}")
print(f"  - Negative Check: {result.negative_check}")

if result.errors:
    print(f"\nErrors: {result.errors}")

print(f"\nResponse (first 500 chars):")
print("-" * 40)
print(result.response[:500])

Test ID: gs-001
Query: What is our remote work policy?
Passed: True
Tools Used: ['vector_search']

Checks:
  - Tool Check: True
  - Source Check: True
  - Content Check: True
  - Negative Check: True

Response (first 500 chars):
----------------------------------------
Acme Corp Remote Work Policy (effective Jan 1, 2024; last updated Mar 15, 2024)

- Remote-first: Employees may work fully remote, in-office, or hybrid based on preference and role requirements.
- Core hours (availability expected): 10:00 AM – 3:00 PM Pacific Time, Monday–Friday. Outside core hours, you can structure your day flexibly.
- Home office support:
  - Equipment provided: laptop (refreshed every 3 years), one 27" monitor, keyboard/mouse, headset.
  - Home office stipend: $500 annually (


## 🔬 Deep Dive: How Each Check Works

Each test case runs multiple independent checks. Understanding these helps you write better tests.

### The Four Types of Checks:

| Check | What It Verifies | Why It Matters |
|-------|-----------------|----------------|
| **Tool Check** | Correct tools were used | Agent should route queries appropriately |
| **Source Check** | Correct documents were cited | Agent should find relevant information |
| **Content Check** | Key facts appear in response | Agent should extract and relay correct info |
| **Negative Check** | Failure phrases are absent | Agent shouldn't hallucinate or give up |

Let's see each check in action:

In [5]:
# Tool check example
expected_tools = ["vector_search"]
actual_tools = ["vector_search"]

passed, error = check_tools(expected_tools, actual_tools)
print(f"Tool Check: {'PASS' if passed else 'FAIL'}")
print(f"Expected: {expected_tools}")
print(f"Actual: {actual_tools}")

# Try with wrong tools
actual_wrong = ["sql_query"]
passed, error = check_tools(expected_tools, actual_wrong)
print(f"\nWrong Tool Check: {'PASS' if passed else 'FAIL'}")
print(f"Error: {error}")

Tool Check: PASS
Expected: ['vector_search']
Actual: ['vector_search']

Wrong Tool Check: FAIL
Error: Missing tools: ['vector_search']


In [6]:
# Content check example
keywords = ["remote", "core hours", "500"]
response = "Our remote work policy includes core hours from 10 AM to 3 PM and a $500 stipend."

passed, error = check_must_contain(keywords, response)
print(f"Content Check: {'PASS' if passed else 'FAIL'}")
print(f"Looking for: {keywords}")
print(f"In response: {response[:100]}...")

# Try with missing keyword
response_missing = "Our remote work policy includes core hours."
passed, error = check_must_contain(keywords, response_missing)
print(f"\nMissing Keyword Check: {'PASS' if passed else 'FAIL'}")
print(f"Error: {error}")

Content Check: PASS
Looking for: ['remote', 'core hours', '500']
In response: Our remote work policy includes core hours from 10 AM to 3 PM and a $500 stipend....

Missing Keyword Check: FAIL
Error: Missing keywords: ['500']


## 🏃 Running Multiple Test Cases

In practice, you run the entire golden set to detect regressions. This is typically done:
- **On every commit** — Part of your CI pipeline
- **Before deployments** — Gate production releases
- **After changes** — Verify nothing broke

Let's run a subset to see what a typical run looks like:

In [7]:
# Run a subset of tests (first 5)
results = []
for tc in test_cases[:5]:
    print(f"Running {tc['id']}: {tc['query'][:40]}...")
    result = run_test_case(tc)
    results.append(result)
    status = "✓ PASS" if result.passed else "✗ FAIL"
    print(f"  {status}")
    if not result.passed:
        for err in result.errors:
            print(f"    - {err}")
    print()

# Summary
passed = sum(1 for r in results if r.passed)
print(f"Results: {passed}/{len(results)} passed")

Running gs-001: What is our remote work policy?...
  ✓ PASS

Running gs-002: How much is the home office stipend?...
  ✓ PASS

Running gs-003: What is our PTO policy?...
  ✓ PASS

Running gs-004: How do I handle a production incident?...
  ✓ PASS

Running gs-005: What are our pricing tiers?...
  ✓ PASS

Results: 5/5 passed


## ✍️ Writing Effective Test Cases

A good golden set test case should be:

### 1. **Specific and Verifiable**
```yaml
# ❌ Bad: Too vague
must_contain: ["policy"]

# ✅ Good: Specific, verifiable facts
must_contain: ["30-day", "refund", "annual plans"]
```

### 2. **Representative of Real Queries**
Base tests on actual user questions, not hypothetical edge cases (save those for Stage 2).

### 3. **Include Negative Checks**
Catch hallucinations and false negatives:
```yaml
must_not_contain: ["I don't know", "I cannot", "no information"]
```

Let's create and test a new case:

In [ ]:
# Define a new test case
new_test = {
    "id": "gs-custom-001",
    "category": "vector_search",
    "query": "What are the code review requirements?",
    "expected_tools": ["vector_search"],
    "expected_sources": ["code_review_standards.md"],
    "must_contain": ["review", "approval"],
    "must_not_contain": ["I don't know"],
}

# Run it
print("Testing new case:")
print(f"Query: {new_test['query']}")
print()

result = run_test_case(new_test)
print(f"Passed: {result.passed}")
print(f"Tools used: {result.tools_used}")
if result.errors:
    print(f"Errors: {result.errors}")

Testing new case:
Query: What are the code review requirements?



## 📊 Best Practices Checklist

| Category | Recommendation |
|----------|---------------|
| **Size** | 10-50 test cases (quality over quantity) |
| **Run time** | Should complete in < 5 minutes |
| **Coverage** | At least 2-3 cases per tool/capability |
| **Updates** | Add cases when you find production bugs |

### Anti-Patterns to Avoid

❌ **Testing exact wording** — AI responses vary, test for meaning  
❌ **Too many test cases** — Diminishing returns, slow runs  
❌ **Vague must_contain** — "answer" is not specific enough  
❌ **No negative checks** — You'll miss hallucinations  

---

## 🎓 Key Takeaways

1. **Golden sets are your foundation** — Without them, you don't know if your agent works
2. **They test constraints, not exact outputs** — Use `must_contain`, `expected_tools`, etc.
3. **Keep them small and fast** — 10-50 cases that run in minutes
4. **Run on every change** — They're your first line of defense
5. **Base them on real queries** — Use actual user questions from production

---

## ⏭️ What's Next?

Golden sets tell you if things are broken, but they don't tell you about **coverage**. 

- Are you testing all the different types of queries users ask?
- What about edge cases and ambiguous questions?
- How do you know which categories need more testing?

That's what **Stage 2: Labeled Scenarios** addresses:

```bash
cd ../stage_2_labeled_scenarios
uv run jupyter notebook walkthrough.ipynb
```